## 重点代码题目

## 数据结构设计
- 设计一个数据结构，完成set(key,value),get(key),set_all_value(value)，其中最后一个函数时间复杂度小于O（n）【O（1）】

```
考虑到普通set和set_all_value 需要比较先后顺序。
方法1，基于更新时间戳（版本）
    两类值，k上的值和 global的值，每个index和glabal值都维护更新时间戳
    get(index)的时候，如果global值的时间大于index上的时间，取global的值，否则取index上的值


方法2 exclude list (hash set) 
    setAll的时候 对global赋值，同时清空exclude list
    之后如果有set(index) 加到exclude list
    get(index)的时候
    如果全局值为null，没啥好说的，取index上的值
    否则，看index是否在exclude list里 如果在，取index上的，否则取global值
```

In [3]:
class KV(object):
    def __init__(self):
        self.count = 0
        self.all_count = -1
        self.all_value = None
        self.map = dict()
    
    def set(self, k, v):
        self.map[k] = (v, self.count)
        self.count += 1
    
    def get(self, k):
        if k in self.map:
            if self.map[k][-1] > self.all_count:
                return self.map[k][0]
            else:
                return self.all_value
        return 
    
    def set_all_value(self, v):
        self.all_value = v
        self.all_count = self.count
        self.count += 1 

In [7]:
class timerMap(object):
    def __init__(self):
        self.time = 0
        self.all_value = None
        self.all_value_time = -1
        self.map = {}
    
    def set_value(self, k, v):
        self.map[k] = (v, self.time)
        self.time += 1
    
    def set_all_value(self, v):
        self.all_value = v
        self.all_value_time = self.time
        self.time += 1
    
    def get(self, k):
        if k in self.map:
            if not self.all_value or self.map[k][-1] > self.all_value_time:
                return self.map[k][0]
            else:
                return self.all_value

In [8]:
mp = timerMap()

In [10]:
for i in range(5):
    mp.set_value(i, str(i))
mp.set_all_value(6)
mp.set_value(3, 9999)

for i in range(5):
    print(i, mp.get(i))

0 6
1 6
2 6
3 9999
4 6


## 261. 以图判树 - 锁
- 给定从 0 到 n-1 标号的 n 个结点，和一个无向边列表（每条边以结点对来表示），请编写一个函数用来判断这些边是否能够形成一个合法有效的树结构。

```
示例 1：
输入: n = 5, 边列表 edges = [[0,1], [0,2], [0,3], [1,4]]
输出: true
    
示例 2:
输入: n = 5, 边列表 edges = [[0,1], [1,2], [2,3], [1,3], [1,4]]
输出: false
    
注意：你可以假定边列表 edges 中不会出现重复的边。由于所有的边是无向边，边 [0,1] 和边 [1,0] 是相同的，因此不会同时出现在边列表 edges 中。
```


In [8]:
class unionFindSet(object):
    def __init__(self, n):
        self.parent = [i for i in range(n)]
        self.count = n
        
    def find(self, p):
        root = p
        while root != self.parent[root]:
            root = self.parent[root]
        while p != self.parent[p]:
            x = p
            self.parent[x] = root
            p = self.parent[p]
        return root
    
    def union(self, p, q):
        p1 = self.find(p)
        p2 = self.find(q)
        if p1 == p2:
            return False
        self.parent[p] = q
        self.count -= 1
        return True

In [9]:
def isValidTree(n, edges):
    if n < 2:
        return True
    ufset = unionFindSet(n)
    
    for x, y in edges:
        if not ufset.union(x, y):
            return False
    return True
    

In [10]:
n = 5
edges = [[0,1], [0,2], [0,3], [1,4]]
isValidTree(n, edges)

True

In [11]:
edges = [[0,1], [1,2], [2,3], [1,3], [1,4]]
isValidTree(n, edges)

False

## 130 被围绕的区域

In [2]:
from typing import List

In [30]:
class Solution:
    def solve(self, board: List[List[str]]) -> None:
        """
        Do not return anything, modify board in-place instead.
        """
        class unionFind(object):
            def __init__(self, n):
                self.parent = [_ for _ in range(n)]
                self.count = n
            
            def find(self, p):
                root = p
                while root != self.parent[root]:
                    root = self.parent[root] 
                while p != self.parent[p]:
                    x = p
                    self.parent[x] = root
                    p = self.parent[p]
                return root
            
            def union(self, p, q):
                p1 = self.find(p)
                p2 = self.find(q)
                if p1 == p2:
                    return
                self.parent[p1] = p2
                self.count -= 1
        
        if not board or not board[0]:
            return board
        drec = [(1, 0), (0, 1), (0, -1), (-1, 0)]
        r = len(board)
        c = len(board[0])
        uf = unionFind(r*c + 1)

        for i in range(r):
            for j in range(c):
                if board[i][j] == "O":
                    if i == 0 or j == 0 or i == r-1 or j == c-1:
                        uf.union(i*c + j, r*c)
                    for x in drec:
                        i_x = i + x[0]
                        j_x = j + x[1]
                        if 0 <= i_x < r and 0 <= j_x < c and board[i_x][j_x] == "O":
                            uf.union(i_x*c + j_x, i*c + j)
        for i in range(r):
            for j in range(c):
                if uf.find(i*c + j) == uf.find(r*c):
                    board[i][j] = "O"
                else:
                    board[i][j] = "X"
        return board

In [31]:
board = [["X","O","X","X"],
 ["O","X","O","X"],
 ["X","O","X","O"],
 ["O","X","O","X"]]

# [["X","O","X","X"],
#  ["O","X","O","X"],
#  ["X","O","X","O"],
#  ["O","X","O","X"]]

# 预期：
# [["X","O","X","X"],
#  ["O","X","X","X"],
#  ["X","X","X","O"],
#  ["O","X","O","X"]]

In [32]:
Solution().solve(board)

[['X', 'O', 'X', 'X'],
 ['O', 'X', 'X', 'X'],
 ['X', 'X', 'X', 'O'],
 ['O', 'X', 'O', 'X']]

In [33]:
board = [["X","O","X","O","X","O","O","O","X","O"],
         ["X","O","O","X","X","X","O","O","O","X"],
         ["O","O","O","O","O","O","O","O","X","X"],
         ["O","O","O","O","O","O","X","O","O","X"],
         ["O","O","X","X","O","X","X","O","O","O"],
         ["X","O","O","X","X","X","O","X","X","O"],
         ["X","O","X","O","O","X","X","O","X","O"],
         ["X","X","O","X","X","O","X","O","O","X"],
         ["O","O","O","O","X","O","X","O","X","O"],
         ["X","X","O","X","X","X","X","O","O","O"]]

In [34]:
Solution().solve(board)

[['X', 'O', 'X', 'O', 'X', 'O', 'O', 'O', 'X', 'O'],
 ['X', 'O', 'X', 'X', 'X', 'X', 'O', 'O', 'O', 'X'],
 ['O', 'O', 'X', 'X', 'X', 'X', 'O', 'O', 'X', 'X'],
 ['O', 'O', 'X', 'X', 'X', 'X', 'X', 'O', 'O', 'X'],
 ['O', 'O', 'X', 'X', 'X', 'X', 'X', 'O', 'O', 'O'],
 ['X', 'O', 'O', 'X', 'X', 'X', 'X', 'X', 'X', 'O'],
 ['X', 'O', 'X', 'X', 'X', 'X', 'X', 'O', 'X', 'O'],
 ['X', 'X', 'O', 'X', 'X', 'X', 'X', 'O', 'O', 'X'],
 ['O', 'O', 'O', 'O', 'X', 'X', 'X', 'O', 'X', 'O'],
 ['X', 'X', 'O', 'X', 'X', 'X', 'X', 'O', 'O', 'O']]

## 数组排序

In [12]:
def quick_sort(array, left, right):
    if left < right:
        mid = partition(array, left, right)
        quick_sort(array, left, mid-1)
        quick_sort(array, mid+1, right)

def partition(array, left, right):
    tmp = array[left]
    while left < right:
        while left < right and array[right] >= tmp:
            right -= 1
        array[left] = array[right]
        while left < right and array[left] <= tmp:
            left += 1
        array[right] = array[left]
    array[left] = tmp
    return left

In [62]:
import random

In [64]:
li = list(range(50))
random.shuffle(li)
print(li)

[6, 0, 45, 36, 24, 19, 32, 28, 9, 22, 48, 43, 17, 27, 1, 38, 5, 44, 40, 37, 26, 15, 42, 23, 29, 20, 30, 31, 10, 33, 16, 14, 25, 3, 7, 39, 12, 41, 21, 4, 35, 13, 11, 46, 47, 18, 8, 2, 34, 49]


In [16]:
quick_sort(li, 0, len(li)-1)

In [18]:
print(li)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]


##  归并排序

In [60]:
def merge(array, left, mid, right):
    i = left
    j = mid + 1
    tmp = []
    while i <= mid and j <= right:
        if array[i] <= array[j]:
            tmp.append(array[i])
            i += 1
        else:
            tmp.append(array[j])
            j += 1
    while i <= mid:
        tmp.append(array[i])
        i += 1
    while j <= right:
        tmp.append(array[j])
        j += 1
    array[left: right+1] = tmp

def merge_sort(array, left, right):
    if left < right:
        mid = (left + right) // 2
        merge_sort(array, left, mid)
        merge_sort(array, mid+1, right)
        merge(array, left, mid, right)

In [65]:
random.shuffle(li)
print(li)

[2, 17, 31, 49, 10, 11, 36, 44, 35, 14, 15, 30, 46, 9, 33, 47, 26, 0, 38, 6, 25, 19, 22, 34, 21, 16, 12, 42, 48, 20, 37, 13, 40, 24, 32, 43, 5, 3, 18, 28, 27, 8, 23, 39, 4, 29, 41, 45, 1, 7]


In [66]:
merge_sort(li, 0, len(li)-1)
print(li)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]


In [67]:
def sift(array, left, right):
    i = left 
    j = 2 * i + 1 
    tmp = array[i]
    while j <= right:
        while j < right and array[j+1] > array[j]:
            j = j + 1
        if array[j] > tmp:
            array[i] = array[j]
            i = j
            j = 2*i + 1
        else:
            break
    array[i] = tmp

def heap_sort(array):
    # 建堆
    n = len(array)
    for i in range(n//2-1, -1, -1):
        sift(array, i, n-1)
    
    for i in range(n-1, -1, -1):
        array[0], array[i] = array[i], array[0]
        sift(array, 0, i-1)

In [69]:
random.shuffle(li)
print(li)

[10, 2, 26, 21, 27, 33, 43, 34, 48, 44, 6, 49, 20, 12, 38, 5, 3, 24, 19, 41, 7, 35, 14, 32, 40, 0, 11, 31, 18, 36, 4, 37, 17, 30, 15, 46, 22, 47, 13, 25, 29, 23, 8, 1, 16, 42, 28, 39, 9, 45]


In [70]:
heap_sort(li)
print(li)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]


In [29]:
def sift(array, left, right):
    i = left # 根节点
    j = 2*i + 1 # 根的左孩子
    tmp = array[i]
    while j <= right:
        if j < right and array[j] < array[j+1]:
            j = j + 1
        if tmp < array[j]:
            array[i] = array[j]
            i = j
            j = 2*i + 1
        else:
            break
    array[i] = tmp
    
def heap_sort(array):
    n = len(array)
    for i in range(n//2 -1, -1, -1):
        sift(array, i, n-1)
    for i in range(n-1, -1, -1):
        array[0], array[i] = array[i], array[0]
        sift(array, 0, i-1)

In [30]:
random.shuffle(li)
print(li)

[40, 26, 2, 22, 29, 48, 1, 21, 11, 39, 43, 41, 31, 0, 15, 35, 49, 9, 14, 20, 3, 38, 10, 45, 18, 19, 24, 27, 28, 7, 36, 47, 16, 46, 34, 6, 4, 23, 8, 32, 37, 12, 42, 13, 30, 5, 44, 17, 33, 25]


In [31]:
heap_sort(li)
print(li)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]


## 链表

In [1]:
class LinkNode(object):
    def __init__(self, v=None):
        self.value = v
        self.next = None
    
    def __repr__(self):
        r = []
        cur = self
        while cur:
            r.append(cur.value)
            cur = cur.next
        return "<Link: [{}]>".format(", ".join([str(s) for s in r]))

In [2]:
def p(root):
    cur = root
    r = []
    while cur:
        print("cur:", cur, cur.value)
        r.append(cur.value)
        cur = cur.next
    return r

In [3]:
def reverse(head):
    #pre = LinkNode() # 这种写法是错误的，会造成死循环
    pre = None
    cur = head
    while cur:
        next_ = cur.next
        cur.next = pre
        pre = cur
        cur = next_
    return pre

In [4]:
def buildLink(li):
    pre = cur = LinkNode()
    for i in li:
        cur.next = LinkNode(i)
        cur = cur.next
    return pre.next

In [5]:
head = buildLink(list(range(10)))

In [6]:
head

<Link: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]>

In [7]:
newhead = reverse(head)

In [8]:
newhead

<Link: [9, 8, 7, 6, 5, 4, 3, 2, 1, 0]>

In [17]:
class Solution:
    def reorderList(self , head ):
        # write code here
        def reverse(head):
            pre = None
            cur = head
            while cur:
                nxt = cur.next
                cur.next = pre
                pre = cur
                cur = nxt
            return pre
        if not head or not head.next:
            return head
        slow = head
        fast = head
        while fast and fast.next:
            slow = slow.next
            fast = fast.next.next
        p1 = head
        p = slow.next
        slow.next = None
        p2 = reverse(p)
        pre = LinkNode(-1)
        new_head = pre
        count = 1
        while p1 and p2:
            if count % 2 == 1:
                pre.next = p1
                p1 = p1.next
            else:
                pre.next = p2
                p2 = p2.next
            count += 1
            pre = pre.next
        if p1:
            pre.next = p1
        else:
            pre.next = p2
        return new_head.next

In [18]:
head = buildLink([])

In [19]:
x = Solution().reorderList(head)

In [20]:
p(x)

[]

## K个一组反转
-给定一个单链表的头节点 head,实现一个调整单链表的函数，使得每K个节点之间为一组进行逆序，并且从链表的尾部开始组起，头部剩余节点数量不够一组的不需要逆序。（不能使用队列或者栈作为辅助）
- [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
- [0, 3, 2, 1, 6, 5, 4, 9, 8, 7]

In [13]:
def reverseLinkByKGroup(head, k):
    def reverseKnode(head, k):
        pre = None
        cur = head
        for i in range(k):
            next_ = cur.next
            cur.next = pre
            pre = cur
            cur = next_
        head.next = cur
        return pre 
    
    if not head or not head.next or k==1:
        return head
    n = 0
    cur = head
    while cur:
        n += 1
        cur = cur.next
    
    start = n % k
    
    new_head = pre = LinkNode()
    pre.next = head
    for _ in range(start):
        pre = pre.next
    for i in range(n):
        if (i-start) % k == 0 and n-i >= k:
#             print(pre.value)
            rhead = reverseKnode(pre.next, k)
            pre.next = rhead
            for _ in range(k):
                if pre:
                    pre = pre.next
    return new_head.next

In [10]:
head = buildLink(list(range(10)))

In [11]:
head

<Link: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]>

In [15]:
nhead = reverseLinkByKGroup(head, 3)
nhead

<Link: [0, 3, 2, 1, 6, 5, 4, 9, 8, 7]>

## topK

In [26]:
class Solution:
    def GetLeastNumbers_Solution(self, tinput, k):
        # write code here
        def sift(array, left, right):
            i = left
            j = 2 * i + 1
            tmp = array[i]
            while j <= right:
                if j < right and array[j+1] > array[j]:
                    j = j + 1
                if tmp <= array[j]:
                    array[i] = array[j]
                    i = j
                    j = 2*i + 1
                else:
                    break
            array[i] = tmp
        n = len(tinput)
        if k > n :
            return []
        for i in range(n//2 - 1, -1, -1):
            sift(tinput, i, n-1)
        res = []
        for i in range(n-1, n-k-1, -1):
            tinput[0], tinput[i] = tinput[i], tinput[0]
            sift(tinput, 0, i-1)
            if i == k:
                break
        return tinput[-k :]

In [27]:
tinput = [4,5,1,6,2,7,3,8]
k = 4

In [28]:
Solution().GetLeastNumbers_Solution(tinput, k)

[5, 6, 7, 8]

## topK
- 有一个整数数组，请你根据快速排序的思路，找出数组中第K大的数。
- 给定一个整数数组a,同时给定它的大小n和要找的K(K在1到n之间)，请返回第K大的数，保证答案存在。

In [88]:
def partition(arr, left, right):
    tmp = arr[left]
    while left < right:
        while left < right and tmp >= arr[right]:
            right -= 1
        arr[left] = arr[right]
        while left < right and tmp <= arr[left]:
            left += 1
        arr[right] = arr[left]
    arr[left] = tmp
    return left

def quickSortTopK(arr, left, right, K):
    if left < right:
        p = partition(arr, left, right)
        if p - left == K-1 :
            return arr[p]
        elif p - left > K-1:
            return quickSortTopK(arr, left, p-1, K)
        else:
            return quickSortTopK(arr, p+1, right, K-p+left-1)
    # 这个很重要，没有的话会有bug
    return arr[left]
        
class Solution:
    def findKth(self, a, n, K):
        return quickSortTopK(a, 0, n-1, K)    

In [83]:
import random
li = list(range(10))
random.shuffle(li)
#li = [5, 0, 6, 9, 1, 3, 3, 6, 6, 7, 8, 2, 4]

In [84]:
print(Solution().findKth(li, len(li), 5))

5


In [86]:
li = [363047,1010880,240149,1625009,542307,665705,911808,103856,671544,1300643,1827344,206529,42217,147976,723290,1676726,202900,1364904,638833,797085,1084935,1106701,1339778,641359,268339,1017918,593852,676409,835688,1395103,1435452,63760,902767,938570,442508,1089571,1473932,1672481,288602,1639230,156813,956865,1089083,992310,859686,1004628,1600048,746937,1735047,275727,825630,31172,988291,1740072,1134157,1216362,1691491,554450,993240,1901642,1172064,1724047,699223,3161,1912328,714022,403013,72430,1525111,1088068,1177389,1736044,1421768,43836,1184729,277374,996841,1281388,1445175,1384054,825431,977343,1729999,400418,38737,1175873,1457130,1092726,1364365,1310499,236180,1587696,750808,31565,1568177,996525,817324,511336,307148,56320,1107335,1924246,1364690,307094,324230,915035,1864387,384519,972099,999028,758538,1670952,172914,1014396,1366371,1959283,900176,1213143,388700,1151204,218501,445072,1709306,1266039,991212,9441,733159,996599,1171969,152662,1112686,1755388,1887551,1159458,1880156,807129,1557891,1972221,1034665,156875,1087871,1271922,1717867,1200348,366801,329393,1252011,346961,1612325,778115,1407158,702431,1617464,1055209,679412,255619,501946,1410973,71527,1710248,126647,1494957,247713,88526,373805,1150784,1480820,751880,651477,1364717,1151604,1716583,1244500,1889955,652461,37459,1147266,1201053,198729,1502190,440221,1351202,843466,1514168,1805252,327425,32987,984933,343091,1977846,1856431,1275202,1833003,908675,1147499,54008,294476,49576,1592145,398356,290027,1693781,558998,521352,683953,1304541,417371,1678170,978472,304582,1512909,1841018,474880,1889261,1524851,132764,309932,240652,554940,436798,1579545,965640,1701467,334550,1228603,1460006,1038115,105902,1378983,865756,7147,1180824,676308,99008,1838219,275660,697131,480522,1527571,834530,1355644,574705,413105,35089,659492,512092,1766582,1733740,18889,1070932,1842956,1639344,173217,1775589,1876655,649476,1463287,948251,908713,1738329,82584,646982,1673832,111318,1735799,536460,564199,1027478,777187,371077,1298811,4550,1583634,242715,592559,693205,1894703,659227,904193,1899867,360288,91702,362330,811805,1784400,1509147,1369390,388925,612914,285889,475507,583348,1935562,654035,161934,452682,1551928,431910,289619,352471,879056,1840927,597824,1338566,584757,1178338,30389,122828,1083493,1352813,1443072,420662,195350,483018,860056,1205099,1761118,304240,1227346,1680536,143142,304695,1507819,1591712,157207,1826156,1981284,45207,9410,62955,253932,1135768,1491713,24986,1104017,332532,1210242,668321,666193,756488,1324741,717765,804484,1084670,497411,1042499,1727443,1100728,1024005,900253,815804,1222189,1324316,551277,1758153]
K = 11
print(Solution().findKth(li, len(li), K))

1889955


## 数组数字连续的最长序列（难）

```
给定一个未排序的整数数组 nums ，找出数字连续的最长序列（不要求序列元素在原数组中连续）的长度。
进阶：你可以设计并实现时间复杂度为 O(n) 的解决方案吗？
示例 1：

输入：nums = [100,4,200,1,3,2]
输出：4
解释：最长数字连续序列是 [1, 2, 3, 4]。它的长度为 4。
示例 2：

输入：nums = [0,3,7,2,5,8,4,6,0,1]
输出：9
```

In [91]:
from typing import List

In [92]:
class Solution:
    def longestConsecutive(self, nums: List[int]) -> int:
        dic = dict()
        max_len = 0
        for i in nums:
            if i not in dic:
                left = dic.get(i-1, 0)
                right = dic.get(i+1, 0)
                cur_len = 1 + left + right
                max_len = max(max_len, cur_len)
                dic[i] = cur_len
                dic[i-left] = cur_len
                dic[i+right] = cur_len
        return max_len

## LRU

In [93]:
class DLinkNode(object):
    def __init__(self, k=0, v=0):
        self.key = k
        self.value = v
        self.prev = None
        self.next = None

class MyLRU(object):
    # hash表+双链表
    def __init__(self, capacity):
        self.cache = dict()
        self.head = DLinkNode()
        self.tail = DLinkNode()
        self.head.next = self.tail
        self.tail.prev = self.head
        self.capacity = capacity
        self.size = 0
        
    def get(self, k):
        if k not in self.cache:
            return -1
        node = self.cache[k]
        self.moveToHead(node)
        return node.value
    
    def set(self, k, v):
        if k not in self.cache:
            node = DLinkNode(k, v)
            self.cache[k] = node
            self.addToHead(node)
            self.size += 1
            if self.size > self.capacity:
                # 删除链表尾结点
                node = self.removeTail()
                # 删除映射关系
                self.cache.pop(node.key)
                self.size -= 1
        else:
            node = self.cache[k]
            node.value = v
            self.moveToHead(node)
    
    def addToHead(self, node):
        node.next = self.head.next
        self.head.next.prev = node
        node.prev = self.head
        self.head.next = node
    
    def removeNode(self, node):
        node.prev.next = node.next
        node.next.prev = node.prev
        node.prev = None
        node.next = None
    
    def moveToHead(self, node):
        self.removeNode(node)
        self.addToHead(node)

    def removeTail(self):
        node = self.tail.prev
        self.removeNode(node)
        return node
        
class Solution:
    def LRU(self , operators , k ):
        # write code here
        kv = MyLRU(k)
        res = []
        for li in operators:
            if len(li) == 3:
                kv.set(li[1], li[2])
            elif len(li) == 2:
                res.append(kv.get(li[-1]))
        return res

In [96]:
from collections import OrderedDict

In [122]:
from collections import OrderedDict

class MyLRU(object):
    def __init__(self, capacity):
        self.cache = OrderedDict()
        self.capacity = capacity
        
    def get(self, k):
        if k not in self.cache:
            return -1
        self.cache.move_to_end(k)
        return self.cache[k]
    
    def push(self, k, v):
        self.cache[k] = v
        self.cache.move_to_end(k)
        if len(self.cache) > self.capacity:
            self.cache.popitem(last=False)
        
class Solution:
    def LRU(self , operators , k ):
        # write code here
        kv = MyLRU(k)
        res = []
        for li in operators:
            if len(li) == 3:
                kv.push(li[1], li[2])
            elif len(li) == 2:
                res.append(kv.get(li[-1]))
        return res

In [123]:
operators = [[1,1,1],[1,2,2],[1,3,2],[2,1],[1,4,4],[2,2]]
k = 3

In [124]:
Solution().LRU(operators, k)

[1, -1]

In [114]:
d["1"] = 2

In [115]:
d["2"] = 3

In [116]:
d

OrderedDict([('1', 2), ('2', 3)])

In [117]:
d.popitem(last=False)

('1', 2)

In [118]:
d

OrderedDict([('2', 3)])

In [109]:
d

OrderedDict([('3', None)])

In [5]:
for i in range(0, 5, 2):
    print(i)

0
2
4


In [53]:
import heapq

class medianFinder(object):
    def __init__(self):
        self.count = 0
        self.max_heap = []
        self.min_heap = []
    
    def push(self, x):
        self.count += 1
        heapq.heappush(self.max_heap, (-x, x))
        _, max_n = heapq.heappop(self.max_heap)
        heapq.heappush(self.min_heap, max_n)
        if self.count & 1:
            min_n =  heapq.heappop(self.min_heap)
            heapq.heappush(self.max_heap, (-min_n, min_n))
    
    def getmedian(self):
        if self.count == 0:
            return -1
        if self.count & 1:
            return self.max_heap[0][1]
        else:
            return (self.max_heap[0][1] + self.min_heap[0]) / 2

class Solution:
    def flowmedian(self , operations ):
        # write code here
        res = []
        medianFlow = medianFinder()
        for x in operations:
            if x[0] == 1:
                medianFlow.push(x[-1])
            else:
                m = medianFlow.getmedian()
                res.append(m)
        #print(res)
        return res

In [54]:
op = [[1,5],[2],[1,3],[2],[1,6],[2],[1,7],[2]]
op = [[2]]

In [55]:
Solution().flowmedian(op)

[-1]

In [56]:
[5,4,5,5.5]

[5, 4, 5, 5.5]

In [58]:
#heapq.heapify(q) # 是将 list q 变成一个优先队列；比如滑动窗口中的最大值，首先初始化窗口大小的数据，需要将其变成优先队列

In [59]:
#如果初始列表是空，可以通过 heapq.heappush(q, num) 添加，这样q也是一个优先队列。